In [1]:
!pip install cohere streamlit pdfplumber
!pip install python-dotenv

  Obtaining dependency information for pdfplumber from https://files.pythonhosted.org/packages/e6/c4/d2e09fbc937d1f76baae34e526662cc718e23a904321bf4a40282d190033/pdfplumber-0.11.6-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.8 kB 682.7 kB/s eta 0:00:01
     -------------------------------------- 42.8/42.8 kB 691.8 kB/s eta 0:00:00
  Obtaining dependency information for pdfminer.six==20250327 from https://files.pythonhosted.org/packages/29/2f/409e174b5a0195aa6a814c7359a1285f1c887a4c84aff17ed03f607c06ba/pdfminer_six-20250327-py3-none-any.whl.metadata
  Obtaining dependency information for pypdfium2>=4.18.0 from https://files.pythonhosted.org/packages/a4/f3/8d3a350efb4286b5ebdabcf6736f51d8e3b10dbe68804c6930b00f5cf329/pypdfium2-4.30.1-py3-none-win_amd64.whl.metadata
     ---------------------------------------- 0.0/48.2 kB ? eta -:--:--
     ---------------------------------------- 48.2

In [47]:
%%writefile resume_app_chatbot.py
import streamlit as st
import cohere
import pdfplumber

# Initialize Cohere client
co = cohere.Client("q3dy0ah5ODJBkbHZoNvT5WbSg8TUsbpiFllJaIOs")  # Replace with your actual API key

st.set_page_config(page_title="AI Resume Reviewer", layout="wide")
st.title("AI-Powered Resume Reviewer & Enhancer")

# Sidebar: Resume Upload or Text Input
st.sidebar.header("Upload or Paste Resume")
uploaded_file = st.sidebar.file_uploader("Upload a resume (PDF or DOCX)", type=["pdf", "docx"])
resume_text_area = st.sidebar.text_area("Or paste your resume text:", height=250)

st.sidebar.markdown("---")
st.sidebar.header("Optional")
job_description = st.sidebar.text_area("Paste job description (optional)", height=150)

# Extract resume text
resume_text = ""
if uploaded_file:
    with pdfplumber.open(uploaded_file) as pdf:
        resume_text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
elif resume_text_area.strip():
    resume_text = resume_text_area.strip()

# Enhancement Options
st.subheader("Choose Enhancement Type")
task = st.selectbox(
    "Select what you'd like the assistant to do:",
    [
        "Give overall feedback",
        "Rewrite in a more professional tone",
        "Suitable job roles"
    ]
)

if st.button("Enhance Resume"):
    if not resume_text:
        st.warning("Please upload a resume or paste the text.")
    else:
        prompt = ""
        if task == "Give overall feedback":
            prompt = f"Provide detailed and constructive feedback on the following resume:\n\n{resume_text}"
        elif task == "Rewrite in a more professional tone":
            prompt = f"Rewrite the following resume to sound more polished and professional:\n\n{resume_text}"
        elif task == "Suitable job roles":
            prompt = f"Analyze the following resume and suggest a list of suitable job roles for the candidate:\n\n{resume_text}"
        
        if job_description and task != "Suitable job roles":
            prompt += f"\n\nThe resume should be tailored to this job description:\n{job_description}"

        with st.spinner("Analyzing and enhancing your resume..."):
            response = co.generate(
                model="command-r-plus",
                prompt=prompt,
                max_tokens=10000,
                temperature=0.7
            )
            output_text = response.generations[0].text.strip()

        st.subheader("AI Output")
        st.write(output_text)

        st.download_button(
            label="Download Output",
            data=output_text.encode("utf-8"),
            file_name="resume_output.txt",
            mime="text/plain"
        )

# ----------------------------------
# 🤖 Resume Chatbot Section
# ----------------------------------
st.markdown("---")
st.header("💬 Chat with Resume AI Assistant")

# Initialize chat history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

user_input = st.text_input("Ask a question about your resume, career path, or job suggestions:")

if user_input and resume_text:
    # Build conversation context
    chat_prompt = "You are an expert resume and career advisor AI. Use the following resume to answer the user's questions.\n"
    chat_prompt += f"\nResume:\n{resume_text}\n"
    for pair in st.session_state.chat_history:
        chat_prompt += f"\nUser: {pair['user']}\nAI: {pair['ai']}"
    chat_prompt += f"\nUser: {user_input}\nAI:"

    # Call Cohere
    with st.spinner("Thinking..."):
        chat_response = co.generate(
            model="command-r-plus",
            prompt=chat_prompt,
            max_tokens=10000,
            temperature=0.7
        )
        ai_reply = chat_response.generations[0].text.strip()

    # Update history
    st.session_state.chat_history.append({"user": user_input, "ai": ai_reply})

# Display chat messages
if st.session_state.chat_history:
    for chat in st.session_state.chat_history:
        st.markdown(f"**You:** {chat['user']}")
        st.markdown(f"**AI:** {chat['ai']}")


Overwriting resume_app_chatbot.py


In [48]:
!streamlit run resume_app_chatbot.py

^C
